In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 2. 필요한 패키지 설치
!pip install faiss-cpu sentence-transformers transformers fastapi pyngrok uvicorn fsspec==2025.3.2 transformers datasets accelerate nest_asyncio --quiet

In [ ]:
# 3. 주요 라이브러리 로딩
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, PreTrainedTokenizerFast, GPT2LMHeadModel
import numpy as np
import nest_asyncio
import uvicorn
import faiss
import torch

In [ ]:
# 4. BERT 유사도 검색
embedding_model = SentenceTransformer('jhgan/ko-sbert-sts')
embedding_dim = embedding_model.get_sentence_embedding_dimension()

In [ ]:
# 5. Math 문제 생성 모델 로딩
math_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Math/Problem"
math_problem_tokenizer = AutoTokenizer.from_pretrained(math_problem_model_path)
math_problem_model = AutoModelForCausalLM.from_pretrained(math_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 6. Math 정답 생성 모델 로딩
math_answer_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Math/Answer"
math_answer_tokenizer = AutoTokenizer.from_pretrained(math_answer_model_path)
math_answer_model = AutoModelForCausalLM.from_pretrained(math_answer_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 7. Logic 문제 생성 모델 로딩
logic_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Logic/Problem"
logic_problem_tokenizer = AutoTokenizer.from_pretrained(logic_problem_model_path)
logic_problem_model = AutoModelForCausalLM.from_pretrained(logic_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 8. Logic 정답 생성 모델 로딩
logic_answer_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Logic/Answer"
logic_answer_tokenizer = AutoTokenizer.from_pretrained(logic_answer_model_path)
logic_answer_model = AutoModelForCausalLM.from_pretrained(logic_answer_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 9. Knowledge 문제 생성 모델 로딩
knowledge_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Knowledge/Problem"
knowledge_problem_tokenizer = AutoTokenizer.from_pretrained(knowledge_problem_model_path)
knowledge_problem_model = AutoModelForCausalLM.from_pretrained(knowledge_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 10. Knowledge 정답 생성 모델 로딩
knowledge_answer_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Knowledge/Answer"
knowledge_answer_tokenizer = AutoTokenizer.from_pretrained(knowledge_answer_model_path)
knowledge_answer_model = AutoModelForCausalLM.from_pretrained(knowledge_answer_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 11. Open Problem 문제 생성 모델 로딩
open_problem_model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Open/Problem"
open_problem_tokenizer = AutoTokenizer.from_pretrained(open_problem_model_path)
open_problem_model = AutoModelForCausalLM.from_pretrained(open_problem_model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 12. 문제 생성 함수
def problem(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=512,
        temperature=0.9,
        top_p=0.92,
        top_k=50,
        do_sample=True,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text.split("문제:")[1].strip()

In [ ]:
# 13. 정답 생성 함수
def answer(prompt, model, tokenizer):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=512,
        do_sample=False,
        repetition_penalty=1.0,
        pad_token_id=tokenizer.pad_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text.split("정답:")[1].strip()

In [ ]:
# 14. FastAPI app 설정
app = FastAPI()

class ProblemRequest(BaseModel):
    topic: str
    level: str
    count: int

class ProblemResponse(BaseModel):
    response: list[str]

@app.post("/generate", response_model=ProblemResponse)
def generate_problems(req: ProblemRequest):
    global math_problem_model, math_problem_tokenizer, math_answer_model, math_answer_tokenizer
    global logic_problem_model, logic_problem_tokenizer, logic_answer_model, logic_answer_tokenizer
    global knowledge_problem_model, knowledge_problem_tokenizer, knowledge_answer_model, knowledge_answer_tokenizer
    global open_problem_model, open_problem_tokenizer

    topic = req.topic
    level = req.level
    count = req.count

    max_trials = 5
    threshold = 0.7
    problem_list = []
    trials = 0

    index = faiss.IndexFlatIP(embedding_dim)

    if topic == '수학':
      problem_model = math_problem_model
      problem_tokenizer = math_problem_tokenizer
      answer_model = math_answer_model
      answer_tokenizer = math_answer_tokenizer
    elif topic == '논리':
      problem_model = logic_problem_model
      problem_tokenizer = logic_problem_tokenizer
      answer_model = logic_answer_model
      answer_tokenizer = logic_answer_tokenizer
    elif topic == '상식':
      problem_model = knowledge_problem_model
      problem_tokenizer = knowledge_problem_tokenizer
      answer_model = knowledge_answer_model
      answer_tokenizer = knowledge_answer_tokenizer
    elif topic == '열린문제':
      problem_model = open_problem_model
      problem_tokenizer = open_problem_tokenizer
      answer_model = None
      answer_tokenizer = None
    else:
      return ProblemResponse(response=problem_list)

    with torch.no_grad():
        while len(problem_list) < count and trials < max_trials:
            needed = count - len(problem_list)
            for _ in range(needed):
                problem_prompt = f"난이도: {level} \n문제:"
                problem_result = problem(problem_prompt, problem_model, problem_tokenizer)

                embedding = embedding_model.encode(problem_result, convert_to_numpy=True, normalize_embeddings=True).reshape(1, -1)

                is_duplicate = False
                if index.ntotal > 0:
                    D, I = index.search(embedding, k=1)
                    if D[0][0] >= threshold:
                        is_duplicate = True

                if is_duplicate:
                    continue

                if topic == '열린문제':
                    answer_result = '관점에 따라 다름'
                else:
                    answer_prompt = f"문제: {problem_result} \n정답:"
                    answer_result = answer(answer_prompt, answer_model, answer_tokenizer)

                full_text = f"문제: {problem_result}\n정답: {answer_result}"

                problem_list.append(full_text)
                index.add(embedding)

            trials += 1

    problem_list = problem_list[:count]
    return ProblemResponse(response=problem_list)

In [ ]:
# 15. Ngrok 실행
ngrok.set_auth_token("***")
ngrok.kill()

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)